In [46]:
import cv2
import numpy as np
import handLandmarkMP as hlm
import time
import math
from blunet import connect

In [47]:
hands = hlm.HandDetector()

In [48]:
def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = int(Dx / D)
        y = int(Dy / D)
        return x,y
    else:
        return False

In [4]:
connection = connect("HC-05", "1234")

OSError: The handle is invalid.


In [27]:
def Drive(left_hand):
    middle_tip = np.array(left_hand['middle'][-1][1])
    middle_bot = np.array(left_hand['middle'][0][1])
    if((middle_tip - middle_bot) < 0):
        middle_open = 1
    else:
        middle_open = 0
        
    print(f"middle tip is {middle_tip}")
    print(f"middle bot is {middle_bot}")
    index_tip = np.array(left_hand['index'][-1][1])
    index_bot = np.array(left_hand['index'][0][1])
    if((index_tip - index_bot) < 0):
        index_open = 1
    else:
        index_open = 0
    
    ring_tip = np.array(left_hand['ring'][-1][1])
    ring_bot = np.array(left_hand['ring'][0][1])
    if((ring_tip - ring_bot) < 0):
        ring_open = 1
    else:
        ring_open = 0
    
    pinky_tip = np.array(left_hand['pinky'][-1][1])
    pinky_bot = np.array(left_hand['pinky'][0][1])
    pinky_open = 1 if (pinky_tip - pinky_bot) > 0 else 0
    if((pinky_tip - pinky_bot) < 0):
        pinky_open = 1
    else:
        pinky_open = 0
        
    
    thumb_tip = np.array(left_hand['thumb'][-1][0]) #check by horizontal direction
    thumb_bot = np.array(left_hand['thumb'][0][0])
    if((thumb_tip - thumb_bot) > 0):
        thumb_open = 1
    else:
        thumb_open = 0

    if(not(middle_open and index_open and ring_open and pinky_open) and thumb_open):
        drive = "F"
    elif((middle_open and index_open and ring_open and pinky_open) and not thumb_open):
        drive = "R"
    else:
        drive = "S"
    return drive

In [38]:
def map(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

In [45]:
cap = cv2.VideoCapture(0)
success, img = cap.read()
ref_p0 = np.array((0, img.shape[0]-30))
ref_p1 = np.array((img.shape[1] // 2 - 100, img.shape[0]-30))
ref_p2 = np.array((img.shape[1], img.shape[0]-30))
intersection_point = (ref_p1 + ref_p2) // 2
R_before = 0
R = 0
angle_before = 90
while cap.isOpened():
    success, img = cap.read()
    img = cv2.flip(img, 1)
    if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        continue
    #connection.send(str(angle_before // 2))
    img = hands.findHands(img)
    hands_positions = hands.findPositions(img)
    
    if hands_positions:
        try:
            if(hands_positions['right_hand1']):
                middle = np.array(hands_positions['right_hand1']['middle'][-1]) #get the tip of the middle finger
                img = cv2.circle(img, middle, 0, (255, 255, 255), 10)
                img = cv2.line(img, intersection_point, middle, (255, 255, 255), 2)
                vertical_dist = intersection_point[1] - middle[1]
                horizontal_dist = intersection_point[0] - middle[0]
                hyp = int(np.sqrt((vertical_dist)**2 + (horizontal_dist)**2))
                #print(f"middle is {middle}")
                angle = int(math.degrees(np.arccos(horizontal_dist / hyp)))
                print("angle is {}".format(angle))
                if angle not in range(angle_before-5,angle_before+5) and angle in range(30,150):
                    angle = int(map(angle, 30, 150, 0, 8))
                    print(f"angle sent {angle}")
                    #print(f"angle sent {str(angle // (2 * 10))}")
                    #connection.send(str(angle // (2 * 10)))
                    connection.send(str(angle))
                    angle_before = angle
        except KeyError:
            print("cant find right hand")
            
        try:
            if(hands_positions['left_hand1']):
                drive = Drive(hands_positions['left_hand1'])
                print(f"drive is {drive}")
                print("M"+drive)
                connection.send("M" + drive)
        except KeyError:
            print("cant find left hand")
        
            
        img = cv2.line(img, ref_p1, ref_p2, (0, 0, 0), 2)
        img = cv2.line(img, ref_p0, ref_p2, (0, 0, 0), 2)
        
        #max_angle = np.array((ref_p2[0],ref_p2[1] - int(np.tan(20) * (ref_p2[0] - intersection_point[0]))))
        #max_angle = np.array((ref_p2[0],ref_p2[1] - int(np.sin(20) * 150)))
        #print(f"max angle is {max_angle}")
        
        #img = cv2.line(img, intersection_point, max_angle, (0, 0, 255), 2)
        img = cv2.circle(img, intersection_point, 0, (0, 0, 255), 10)
        
    
    #L1 = line(middle, wrist)
    #L2 = line(ref_p1, ref_p2)
    #R = intersection(L1, L2)
    #if(np.all(np.isclose(R,R_before,atol = 30))):
    
    
    R_before = R
    cv2.imshow("Image", img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

angle is 73
angle sent 2
cant find left hand
angle is 74
angle sent 2
cant find left hand
angle is 74
angle sent 2
cant find left hand
angle is 75
angle sent 3
cant find left hand
angle is 76
angle sent 3
cant find left hand
angle is 77
angle sent 3
cant find left hand
angle is 79
angle sent 3
cant find left hand
angle is 81
angle sent 3
cant find left hand
angle is 83
angle sent 3
cant find left hand
angle is 85
angle sent 3
cant find left hand
angle is 87
angle sent 3
cant find left hand
angle is 87
angle sent 3
cant find left hand
angle is 87
angle sent 3
cant find left hand
angle is 86
angle sent 3
cant find left hand
angle is 84
angle sent 3
cant find left hand
angle is 83
angle sent 3
cant find left hand
angle is 81
angle sent 3
cant find left hand
angle is 80
angle sent 3
cant find left hand
angle is 82
angle sent 3
cant find left hand
angle is 86
angle sent 3
cant find left hand
angle is 91
angle sent 4
cant find left hand
angle is 97
angle sent 4
cant find left hand
angle is 1

In [43]:
connection = connect("HC-05", "1234")

OSError: The handle is invalid.


In [ ]:
int angle = 45;
int length = 150;
Point P1(50,50);
Point P2;

P2.x =  (int)round(P1.x + length * cos(angle * CV_PI / 180.0));
P2.y =  (int)round(P1.y + length * sin(angle * CV_PI / 180.0));